## Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Indian restaurant** in **Dubai**, UAE.

Since there are lots of restaurants in Berlin we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Indian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Dubai center will be obtained using **Google Maps API geocoding** of well known Dubai location (Dubai Mall)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Dubai Mall.

Let's first find the latitude & longitude of Dubai Mall, using specific, well known address and Google Maps geocoding API.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [103]:
import requests

address = 'Dubai Mall'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.1970924 55.2790356376064


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Dubai Mall. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).


In [104]:
search_query = 'Indian'
radius = 5000
print(search_query + ' .... OK!')

Indian .... OK!


In [105]:
CLIENT_ID = 'F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ' # your Foursquare ID
CLIENT_SECRET = 'QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ
CLIENT_SECRET:QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152


In [31]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ&client_secret=QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152&ll=25.1970924,55.2790356376064&v=20180604&query=Indian&radius=5000&limit=30'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d11b463342adf0038b94c23'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'hasPerk': False,
    'id': '5be56ba5646e38002c6fb889',
    'location': {'address': 'Dubai Design District D3',
     'cc': 'AE',
     'city': 'دبي',
     'country': 'الإمارات العربية المتحدة',
     'distance': 2159,
     'formattedAddress': ['Dubai Design District D3',
      'دبي',
      'الإمارات العربية المتحدة'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.18795,
       'lng': 55.297947}],
     'lat': 25.18795,
     'lng': 55.297947,
     'neighborhood': "Za'abeel",
     'state': 'دبي'},
    'name': 'Mohalla Indian Street Food',
    'referralId': 'v-1561441379'},
   {'categories': [{'icon':

In [33]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.state,name,referralId
0,"[{'name': 'Indian Restaurant', 'pluralName': '...",False,5be56ba5646e38002c6fb889,Dubai Design District D3,AE,دبي,الإمارات العربية المتحدة,NaN,2159,"[Dubai Design District D3, دبي, الإمارات العرب...","[{'lng': 55.297947, 'lat': 25.18795, 'label': ...",25.187950,55.297947,Za'abeel,دبي,Mohalla Indian Street Food,v-1561441379
1,"[{'name': 'Indian Restaurant', 'pluralName': '...",False,4b99106ff964a520fb5e35e3,The Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,245,"[The Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'lng': 55.276635557856295, 'lat': 25.1967200...",25.196720,55.276636,NaN,دبي,India Palace Express,v-1561441379
2,"[{'name': 'Asian Restaurant', 'pluralName': 'A...",False,548130c6498e82fa71d319e4,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,1660,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.27153493265844, 'lat': 25.21037743...",25.210377,55.271535,NaN,دبي,Metro Grand Indian Restaurant,v-1561441379
3,"[{'name': 'Indian Restaurant', 'pluralName': '...",False,5a0f13df1755622302f4fe6d,Wasl Square,AE,دبي,الإمارات العربية المتحدة,Safa Park Road,4022,"[Wasl Square (Safa Park Road), دبي, الإمارات ا...","[{'lng': 55.241163, 'lat': 25.185643, 'label':...",25.185643,55.241163,NaN,دبي,Lazeez Indian Cuisine,v-1561441379
4,"[{'name': 'Arts & Crafts Store', 'pluralName':...",False,5016c225e4b0ad6ab3c135f0,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3103,[الإمارات العربية المتحدة],"[{'lng': 55.257954, 'lat': 25.176758, 'label':...",25.176758,55.257954,NaN,NaN,Indian Handy Craft,v-1561441379


In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id
0,Mohalla Indian Street Food,Indian Restaurant,Dubai Design District D3,AE,دبي,الإمارات العربية المتحدة,NaN,2159,"[Dubai Design District D3, دبي, الإمارات العرب...","[{'lng': 55.297947, 'lat': 25.18795, 'label': ...",25.187950,55.297947,Za'abeel,دبي,5be56ba5646e38002c6fb889
1,India Palace Express,Indian Restaurant,The Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,245,"[The Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'lng': 55.276635557856295, 'lat': 25.1967200...",25.196720,55.276636,NaN,دبي,4b99106ff964a520fb5e35e3
2,Metro Grand Indian Restaurant,Asian Restaurant,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,1660,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.27153493265844, 'lat': 25.21037743...",25.210377,55.271535,NaN,دبي,548130c6498e82fa71d319e4
3,Lazeez Indian Cuisine,Indian Restaurant,Wasl Square,AE,دبي,الإمارات العربية المتحدة,Safa Park Road,4022,"[Wasl Square (Safa Park Road), دبي, الإمارات ا...","[{'lng': 55.241163, 'lat': 25.185643, 'label':...",25.185643,55.241163,NaN,دبي,5a0f13df1755622302f4fe6d
4,Indian Handy Craft,Arts & Crafts Store,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3103,[الإمارات العربية المتحدة],"[{'lng': 55.257954, 'lat': 25.176758, 'label':...",25.176758,55.257954,NaN,NaN,5016c225e4b0ad6ab3c135f0
5,Gazebo Indian cuisine DIFC,Indian Restaurant,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,2290,[الإمارات العربية المتحدة],"[{'lng': 55.278635, 'lat': 25.217665, 'label':...",25.217665,55.278635,NaN,NaN,56df0c60498e77585c2fb553
6,The Indian Pavillion,Indian Restaurant,Safa Spinneys,AE,دبي,الإمارات العربية المتحدة,NaN,6045,"[Safa Spinneys, دبي, الإمارات العربية المتحدة]","[{'lng': 55.22764292218067, 'lat': 25.16905078...",25.169051,55.227643,NaN,دبي,4b5beabef964a520261d29e3
7,Indian Movie Night!,Other Nightlife,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,5295,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.23011502071296, 'lat': 25.17967670...",25.179677,55.230115,NaN,دبي,4eeb67842c5beea6206a198e
8,Our Own Indian School,College Academic Building,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3313,[الإمارات العربية المتحدة],"[{'lng': 55.26103777113154, 'lat': 25.17217907...",25.172179,55.261038,NaN,NaN,4f8a78d8e4b055d230f14c1d
9,Indian Barber Shop,Salon / Barbershop,Karama Bus Station 7333,AE,دبي,الإمارات العربية المتحدة,Al Karama Fish Market,5567,[Karama Bus Station 7333 (Al Karama Fish Marke...,"[{'lng': 55.30212161437806, 'lat': 25.24253264...",25.242533,55.302122,NaN,دبي,4f3755dae4b0a1f196cb3b16


In [35]:
dataframe_filtered.name

0                            Mohalla Indian Street Food
1                                  India Palace Express
2                         Metro Grand Indian Restaurant
3                                 Lazeez Indian Cuisine
4                                    Indian Handy Craft
5                            Gazebo Indian cuisine DIFC
6                                  The Indian Pavillion
7                                   Indian Movie Night!
8                                 Our Own Indian School
9                                    Indian Barber Shop
10             Indian passport and visa services center
11                             Jashan Indian Restaurant
12                                        Indian Summer
13                         Little Hut Indian Restaurant
14                                 Sapne Indian Cuisine
15    Pathans South Indian Restaurant, Satwa Round-a...
16                   Temptation Indian Grills & Curries
17                   Global indian international

In [36]:
dataframe_filtered.shape

(20, 15)

In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the 

# add a red circle marker to represent the Dubai Mall
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='JLT',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# add the Indian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(venues_map)


# display map
venues_map

In [109]:
CLIENT_ID = 'F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ' # your Foursquare ID
CLIENT_SECRET = 'QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ
CLIENT_SECRET:QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152


In [110]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ&client_secret=QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152&ll=25.2176571,55.3024292&v=20180604&query=Indian&radius=5000&limit=30'

In [156]:
import requests

address1 = 'Zabeel Park'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location.latitude
longitude1 = location.longitude
print(latitude1, longitude1)

25.23574595 55.29770931689


In [157]:
search_query = 'Indian'
radius1 = 1000
print(search_query + ' .... OK!')

Indian .... OK!


In [167]:

LIMIT1 = 100 # limit of number of venues returned by Foursquare API

radius1 = 2000 # define radius

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius1, LIMIT1)
url

'https://api.foursquare.com/v2/venues/search?client_id=F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ&client_secret=QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152&ll=25.23574595,55.29770931689&v=20180604&query=Indian&radius=2000&limit=100'

In [168]:
results1 = requests.get(url).json()
results1

{'meta': {'code': 200, 'requestId': '5d11ca2c92e7a9002c9dc906'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'hasPerk': False,
    'id': '4bcab216cc8cd13a0c41becf',
    'location': {'address': 'Wafi, Oud Metha',
     'cc': 'AE',
     'city': 'دبي',
     'country': 'الإمارات العربية المتحدة',
     'crossStreet': 'Between Sheikh Rashid Road and 26th Street',
     'distance': 2296,
     'formattedAddress': ['Wafi, Oud Metha (Between Sheikh Rashid Road and 26th Street)',
      'دبي',
      'الإمارات العربية المتحدة'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.229791335703393,
       'lng': 55.31953950687779}],
     'lat': 25.229791335703393,
     'lng': 55.31953950687779,
     'postalCode': '30567',
  

In [169]:
# assign relevant part of JSON to venues
venues1 = results1['response']['venues']

# tranform venues into a dataframe
dataframe1 = json_normalize(venues1)
dataframe1.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'name': 'Indian Restaurant', 'pluralName': '...",False,4bcab216cc8cd13a0c41becf,"Wafi, Oud Metha",AE,دبي,الإمارات العربية المتحدة,Between Sheikh Rashid Road and 26th Street,2296,"[Wafi, Oud Metha (Between Sheikh Rashid Road a...","[{'lng': 55.31953950687779, 'lat': 25.22979133...",25.229791,55.319540,30567,دبي,Asha's - Contemporary Indian Restaurant,v-1561446956
1,"[{'name': 'Indian Restaurant', 'pluralName': '...",False,4c7aab2e97028cfa17dedefe,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,1282,[الإمارات العربية المتحدة],"[{'lng': 55.30536223270538, 'lat': 25.24495199...",25.244952,55.305362,NaN,NaN,Little Hut Indian Restaurant,v-1561446956
2,"[{'name': 'Salon / Barbershop', 'pluralName': ...",False,4f3755dae4b0a1f196cb3b16,Karama Bus Station 7333,AE,دبي,الإمارات العربية المتحدة,Al Karama Fish Market,876,[Karama Bus Station 7333 (Al Karama Fish Marke...,"[{'lng': 55.30212161437806, 'lat': 25.24253264...",25.242533,55.302122,NaN,دبي,Indian Barber Shop,v-1561446956
3,"[{'name': 'Bank', 'pluralName': 'Banks', 'icon...",False,533fbe3411d2ff3074c1c1c2,Mohd. Sultan Belshalat Building,AE,دبي,الإمارات العربية المتحدة,"Community 318, Street 6a",1236,[Mohd. Sultan Belshalat Building (Community 31...,"[{'lng': 55.301937103271484, 'lat': 25.2461738...",25.246174,55.301937,NaN,دبي,Indian Overseas Bank,v-1561446956
4,"[{'name': 'Indian Restaurant', 'pluralName': '...",False,5144a715e4b03dc12f87bdc7,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,1338,[الإمارات العربية المتحدة],"[{'lng': 55.30051805443121, 'lat': 25.24750181...",25.247502,55.300518,NaN,NaN,Indian coffee house,v-1561446956


In [176]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns1]


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1

dataframe_filtered2 = dataframe_filtered1[dataframe_filtered1.categories != 'Indian Restaurant']
dataframe_filtered2

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
2,Indian Barber Shop,Salon / Barbershop,Karama Bus Station 7333,AE,دبي,الإمارات العربية المتحدة,Al Karama Fish Market,876,[Karama Bus Station 7333 (Al Karama Fish Marke...,"[{'lng': 55.30212161437806, 'lat': 25.24253264...",25.242533,55.302122,NaN,دبي,4f3755dae4b0a1f196cb3b16
3,Indian Overseas Bank,Bank,Mohd. Sultan Belshalat Building,AE,دبي,الإمارات العربية المتحدة,"Community 318, Street 6a",1236,[Mohd. Sultan Belshalat Building (Community 31...,"[{'lng': 55.301937103271484, 'lat': 25.2461738...",25.246174,55.301937,NaN,دبي,533fbe3411d2ff3074c1c1c2
6,The Indian High School,High School,Oud Metha,AE,دبي,الإمارات العربية المتحدة,NaN,1879,"[Oud Metha, دبي, الإمارات العربية المتحدة]","[{'lng': 55.31556818954254, 'lat': 25.24065952...",25.240660,55.315568,NaN,دبي,4cef4de93b03f04db41d3edc
7,Indian passport and visa services center,None,Near GPO,AE,الكرامة,الإمارات العربية المتحدة,Umm Hureir Road,1785,"[Near GPO (Umm Hureir Road), الكرامة, الإمارات...","[{'lng': 55.311326, 'lat': 25.246021, 'label':...",25.246021,55.311326,NaN,دبي,4c455c622d3ec9b61e2830ae
8,Indian Visa Centre,None,Karama,AE,NaN,الإمارات العربية المتحدة,NaN,1841,"[Karama, الإمارات العربية المتحدة]","[{'lng': 55.309511, 'lat': 25.248379, 'label':...",25.248379,55.309511,NaN,NaN,4cf33a6de3b9a093f9d94853
10,Indian High School ground,Event Space,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,1846,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.31524276876702, 'lat': 25.24061009...",25.240610,55.315243,NaN,دبي,545ce7cd498e3a2c61dc618f
11,Indian Embassy,Embassy / Consulate,8th St.,AE,دبي,الإمارات العربية المتحدة,NaN,2391,"[8th St., دبي, الإمارات العربية المتحدة]","[{'lng': 55.30917331218128, 'lat': 25.25456304...",25.254563,55.309173,NaN,دبي,4d5778a4e319a1cd31260534
12,Global indian international School,Student Center,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,1514,[الإمارات العربية المتحدة],"[{'lng': 55.28804154010469, 'lat': 25.24616637...",25.246166,55.288042,NaN,NaN,4eedaedcd3e3246bdb9c5e24
17,Mohabbat Indian Night Club,Nightclub,Manhattan Hotel,AE,دبي,الإمارات العربية المتحدة,NaN,1880,"[Manhattan Hotel, دبي, الإمارات العربية المتحدة]","[{'lng': 55.298342227788574, 'lat': 25.2526304...",25.252630,55.298342,NaN,دبي,50a1446de4b09f0186bab59c
18,Mirchi Indian Restaurant,Bar,Khalid Bin Al Waleed Rd,AE,دبي,الإمارات العربية المتحدة,Al karama,2154,"[Khalid Bin Al Waleed Rd (Al karama), دبي, الإ...","[{'lng': 55.31351190224638, 'lat': 25.24880135...",25.248801,55.313512,NaN,دبي,4b0587eef964a5209aa722e3


In [177]:
dataframe_filtered2.name

2                           Indian Barber Shop
3                         Indian Overseas Bank
6                       The Indian High School
7     Indian passport and visa services center
8                           Indian Visa Centre
10                   Indian High School ground
11                              Indian Embassy
12          Global indian international School
17                  Mohabbat Indian Night Club
18                    Mirchi Indian Restaurant
22                                  India Club
Name: name, dtype: object

In [178]:
venues_map1 = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the 

# add a red circle marker to represent the Dubai Mall
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Dubai Mall',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map1)

# add the Indian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(venues_map1)

    # add the Indian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.1
    ).add_to(venues_map1)
    
# display map
venues_map1